# Statistical EDA - Implementation

This notebook demonstrates systematic exploratory data analysis using the univariate → bivariate → multivariate framework. We'll work with the penguins dataset to explore patterns and relationships.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load penguins dataset
penguins = sns.load_dataset('penguins')

print(f"Dataset shape: {penguins.shape}")
print(f"\nFirst few rows:")
penguins.head()

In [ ]:
# Quick overview of variables
penguins.info()

Become a penguin SME:

![penguin species](images/12b-penguins.png)

## Part 1: Univariate Analysis

Univariate analysis examines one variable at a time to understand its distribution and typical values.

### Continuous Variables

In [ ]:
# Distribution of body mass
fig, ax = plt.subplots(figsize=(10, 4))
penguins['body_mass_g'].hist(bins=30, ax=ax, edgecolor='black', alpha=0.7)
ax.set_xlabel('Body Mass (g)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Penguin Body Mass')
plt.tight_layout()
plt.show()


Observation: Right-skewed distribution, most penguins 3000-4500g

Confirm numerically?

In [ ]:
# Summary statistics with interpretation
print("Body Mass Summary Statistics:")
print(penguins['body_mass_g'].describe())

Mean (4202g) > Median (4050g) confirms right skew

In [ ]:
# Boxplot shows quartiles and outliers with data points
fig, ax = plt.subplots(figsize=(8, 5))

# Plot points first (behind)
sns.stripplot(data=penguins, y='body_mass_g', color='steelblue', 
              alpha=0.5, size=5, jitter=True, ax=ax)

# Plot boxplot on top
sns.boxplot(data=penguins, y='body_mass_g', ax=ax, 
            width=0.2, showcaps=True, boxprops=dict(facecolor='none', edgecolor='black', linewidth=1.5),
            whiskerprops=dict(linewidth=1.5), capprops=dict(linewidth=1.5),
            medianprops=dict(color='red', linewidth=2))

ax.set_ylabel('Body Mass (g)')
ax.set_title('Body Mass Distribution with Outliers')
plt.tight_layout()
plt.show()


Several outliers on the heavy side (>5500g)

### Categorical Variables

In [ ]:
# Frequency counts for species
print("Species distribution:")
print(penguins['species'].value_counts())
print("\nObservation: Adelie most common (152), Chinstrap least common (68)")

In [ ]:
# Visual confirmation
fig, ax = plt.subplots(figsize=(8, 5))
penguins['species'].value_counts().plot(kind='bar', ax=ax, color='steelblue', alpha=0.7)
ax.set_xlabel('Species')
ax.set_ylabel('Count')
ax.set_title('Penguin Species Distribution')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.tight_layout()
plt.show()

Key takeaway: We're not just making plots - we're reading them for patterns. What's typical? What's unusual? Does it make sense?

## Part 2: Bivariate Analysis

Bivariate analysis explores relationships between two variables. The approach differs based on variable types.

### Continuous vs Continuous

In [ ]:
# Scatter plot: bill length vs bill depth
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(penguins['bill_length_mm'], penguins['bill_depth_mm'], alpha=0.6)
ax.set_xlabel('Bill Length (mm)')
ax.set_ylabel('Bill Depth (mm)')
ax.set_title('Bill Length vs Bill Depth')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()


Observation: Unclear relationship - maybe weak negative?

In [ ]:
# Quantify with correlation
correlation = penguins['bill_length_mm'].corr(penguins['bill_depth_mm'])
print(f"Correlation: {correlation:.3f}")

Weak negative correlation (-0.235)
But does this tell the whole story?

In [ ]:
# Same scatter but colored by species - reveals Simpson's Paradox
fig, ax = plt.subplots(figsize=(10, 6))
for species in penguins['species'].unique():
    data = penguins[penguins['species'] == species]
    ax.scatter(data['bill_length_mm'], data['bill_depth_mm'], 
               label=species, alpha=0.6, s=50)

ax.set_xlabel('Bill Length (mm)')
ax.set_ylabel('Bill Depth (mm)')
ax.set_title('Bill Length vs Bill Depth by Species')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()


Within each species, the relationship appears POSITIVE

Overall negative correlation is misleading - this is _Simpson's Paradox_: a trend that appears in separate groups of data disappears or reverses when the groups are combined.

Correlation at the aggregate level can completely mislead you about what's actually happening in your data.

![doh](images/12b-doh.png)

Always visualize before trusting correlation!

In [ ]:
# Another example: flipper length vs body mass
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(penguins['flipper_length_mm'], penguins['body_mass_g'], alpha=0.6)
ax.set_xlabel('Flipper Length (mm)')
ax.set_ylabel('Body Mass (g)')
ax.set_title('Flipper Length vs Body Mass')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

correlation = penguins['flipper_length_mm'].corr(penguins['body_mass_g'])
print(f"Correlation: {correlation:.3f}")

Interpretation: Strong positive correlation (0.871)

Clear linear relationship - longer flippers, heavier penguins

### Continuous vs Categorical

Question: Does body mass differ by species?

In [ ]:
# Grouped statistics
print("Body mass by species:")
mass_by_species = penguins.groupby('species')['body_mass_g'].agg(['mean', 'median', 'std'])
print(mass_by_species)

Gentoo clearly heaviest (5076g mean), Adelie lightest (3701g)

Does this fit expectations?

In [ ]:
# Box plots for visual comparison
fig, ax = plt.subplots(figsize=(10, 6))
penguins.boxplot(column='body_mass_g', by='species', ax=ax)
ax.set_xlabel('Species')
ax.set_ylabel('Body Mass (g)')
ax.set_title('Body Mass by Species')
plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()


Distributions separate clearly - Gentoo penguins are much larger

We'd need formal statistical tests (like ANOVA) to confirm this difference is statistically significant

### Categorical vs Categorical

Question: Are species distributed evenly across islands?

`pd.crosstab` to the rescue!

Cross-tabulation creates a frequency table showing how many observations fall into each combination of categories. It's like a two-way frequency count - rows are one variable, columns are another, cells show counts.

In [ ]:
# Cross-tabulation
crosstab = pd.crosstab(penguins['species'], penguins['island'])
print("Species by Island:")
print(crosstab)

Observation: Chinstrap ONLY on Dream island, Gentoo ONLY on Biscoe.

Adelie on all three islands.

Visualize...

In [ ]:
# Visualize the joint distribution
fig, ax = plt.subplots(figsize=(10, 6))
crosstab.plot(kind='bar', stacked=True, ax=ax, alpha=0.8)
ax.set_xlabel('Species')
ax.set_ylabel('Count')
ax.set_title('Species Distribution Across Islands')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend(title='Island')
plt.tight_layout()
plt.show()

### Key Questions for Bivariate Analysis

After exploring relationships, ask:

- Is there a relationship? (visual + quantitative)
- How strong is it?
- Is it linear or non-linear?
- Does it hold across subgroups? (check for Simpson's Paradox, avoid Doh! moments)
- What does it suggest? (correlation ≠ causation, but it generates hypotheses)

## Part 3: Multivariate Analysis

Multivariate analysis looks at patterns across many variables simultaneously.

### Correlation Matrix

In [ ]:
# Calculate correlations for all numeric variables
numeric_cols = penguins.select_dtypes(include=[np.number]).columns
corr_matrix = penguins[numeric_cols].corr()

print("Correlation Matrix:")
print(corr_matrix.round(3))

Hard to read / interpret. Visualize!

In [ ]:
# Heatmap visualization
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, ax=ax, cbar_kws={'label': 'Correlation'})
ax.set_title('Correlation Matrix: All Numeric Variables')
plt.tight_layout()
plt.show()

Strong positive correlations:
  - Flipper length ↔ Body mass (0.87): Larger penguins have longer flippers
  - Bill length ↔ Flipper length (0.66): General size relationship
  - Bill length ↔ Body mass (0.60): Larger bills on larger penguins

Weak/negative correlations:
  - Bill length ↔ Bill depth (-0.24): Simpson's Paradox we saw earlier

Pattern: Most measurements correlate positively (larger penguins are larger overall)

### Pair Plots

A pair plot gives you a comprehensive overview of all relationships between numeric variables in one visualization. Instead of making scatter plots one pair at a time (tedious), the pair plot creates them all automatically in a grid.

Useful for quick exploration, pattern discovery, hypothesis generation, sanity check. Best used early in multivariate exploration to get the "big picture" for < 10 numeric variables.

In [ ]:
# Create pair plot without species coloring
sns.pairplot(penguins, vars=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'])
plt.suptitle('Pair Plot: All Numeric Variables', y=1.01)
plt.tight_layout()
plt.show()

Reading the pair plot:
  - Diagonal: Distribution of each variable (univariate)
  - Off-diagonal: All pairwise relationships (bivariate)

In [ ]:
# Enhanced with species coloring
sns.pairplot(penguins, vars=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
             hue='species', height=2)
plt.suptitle('Pair Plot Colored by Species', y=1.01)
plt.tight_layout()
plt.show()

Key insight: Species separate clearly in multivariate space
  - Gentoo: Largest, longest flippers, intermediate bills
  - Adelie: Smallest, shortest bills and flippers
  - Chinstrap: Intermediate size, longest bills

Clusters visible across multiple dimensions - species is a strong predictor

### Multivariate Patterns

From our multivariate analysis:

- Body measurements cluster together (flipper, mass, bill length all correlate)
- Species create natural groupings visible across multiple dimensions
- Bill depth behaves differently (Simpson's Paradox effect)
- No extreme multivariate outliers (points that are normal individually but weird together)

## Part 4: Data Transformations for Analysis

Sometimes we need to transform variables to answer specific questions. Here we'll see binning and encoding in analytical context.

### Binning Continuous Variables

Motivation: We want to analyze bill length by body mass category, but body mass is continuous. Solution: create categories.

Binning (discretization) converts continuous variables into categories by dividing the range into intervals (bins).

In Pandas:

- `pd.cut()`: Creates equal-width bins (e.g., 0-33%, 33-66%, 66-100% of the range)
- `pd.qcut()`: Creates equal-frequency bins (e.g., quartiles - each bin has ~same number of observations)


In [ ]:
# Create equal-width bins with cut()
penguins_binned = penguins.copy()
penguins_binned['mass_category'] = pd.cut(penguins_binned['body_mass_g'], 
                                            bins=3, 
                                            labels=['Light', 'Medium', 'Heavy'])

print("Equal-width bins (cut):")
print(penguins_binned['mass_category'].value_counts().sort_index())
print("\nObservation: Unequal counts - most penguins are Light or Medium")

In [ ]:
# Create equal-frequency bins with qcut()
penguins_binned['mass_quartile'] = pd.qcut(penguins_binned['body_mass_g'], 
                                             q=4, 
                                             labels=['Q1', 'Q2', 'Q3', 'Q4'])

print("Equal-frequency bins (qcut):")
print(penguins_binned['mass_quartile'].value_counts().sort_index())
print("\nObservation: Roughly equal counts in each quartile")

In [ ]:
# Compare the two approaches visually
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# cut() bins
penguins_binned['mass_category'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='steelblue', alpha=0.7)
axes[0].set_title('cut(): Equal Width Bins')
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# qcut() bins
penguins_binned['mass_quartile'].value_counts().sort_index().plot(kind='bar', ax=axes[1], color='coral', alpha=0.7)
axes[1].set_title('qcut(): Equal Frequency Bins')
axes[1].set_xlabel('Quartile')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

When to use which:
  - cut(): When bin width matters (e.g., age ranges 0-20, 20-40, 40-60)
  - qcut(): When you want equal-sized groups (e.g., quartiles, deciles)

### Encoding Categorical Variables

Motivation: We want numeric representation of species for certain analyses (like including in correlation matrix).

*One-hot encoding* converts categorical variables into numeric format by creating binary (0/1) columns for each category. Each observation gets a 1 in the column for its category, 0s elsewhere.

Method: `pd.get_dummies(series, prefix='name')` creates these binary columns automatically.

Why? Some analyses require numeric representation of categories (like correlation matrices). This is covered in depth in INSY 7120.

In [ ]:
# One-hot encoding with get_dummies()
species_encoded = pd.get_dummies(penguins['species'], prefix='species')
print("One-hot encoded species:")
print(species_encoded.head())
print("\nEach species gets a binary (0/1) column")

In [ ]:
# Combine with original data
penguins_encoded = pd.concat([penguins, species_encoded], axis=1)
print("Original + encoded columns:")
print(penguins_encoded[['species', 'species_Adelie', 'species_Chinstrap', 'species_Gentoo']].head(10))

Dummy variable trap: If you include all categories as separate columns, they're perfectly correlated (if not Adelie and not Chinstrap → must be Gentoo). This causes problems in statistical models. Drop one category to avoid this.

In [ ]:
# Dummy variable trap: drop one category
species_encoded_safe = pd.get_dummies(penguins['species'], prefix='species', drop_first=True)
print("With drop_first=True (avoiding dummy variable trap):")
print(species_encoded_safe.head())

### Transformation in Context

Complete example: Does bill shape (length/depth ratio) differ by body mass category?

In [ ]:
# Create bill ratio
penguins_transformed = penguins.copy()
penguins_transformed['bill_ratio'] = (penguins_transformed['bill_length_mm'] / 
                                       penguins_transformed['bill_depth_mm'])

# Create mass categories
penguins_transformed['mass_category'] = pd.cut(penguins_transformed['body_mass_g'], 
                                                 bins=3, 
                                                 labels=['Light', 'Medium', 'Heavy'])

# Analyze
print("Bill ratio by body mass category:")
ratio_by_mass = penguins_transformed.groupby('mass_category', observed=False)['bill_ratio'].agg(['mean', 'median', 'count'])
print(ratio_by_mass)

In [ ]:
# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
penguins_transformed.boxplot(column='bill_ratio', by='mass_category', ax=ax)
ax.set_xlabel('Body Mass Category')
ax.set_ylabel('Bill Ratio (Length/Depth)')
ax.set_title('Bill Shape by Body Mass Category')
plt.suptitle('')
plt.tight_layout()
plt.show()

Interpretation: Heavier penguins have higher bill ratios (relatively longer bills)

This is the pattern: transform → analyze → interpret

## Part 5: Putting It Together

### The Systematic EDA Workflow

When approaching any dataset:

1. Univariate: Understand each variable individually
2. Bivariate: Explore relationships relevant to your questions
3. Multivariate: Look for patterns across many variables
4. Transform as needed for specific analyses
5. Document findings and iterate

### Example: What Predicts Body Mass?

In [ ]:
# 1. Univariate: What's the distribution of body mass?
print("Step 1 - Univariate Analysis:")
print(penguins['body_mass_g'].describe())
print("Distribution: Right-skewed, range 2700-6300g")

In [ ]:
# 2. Bivariate: What's associated with body mass?
print("\nStep 2 - Bivariate Analysis:")
print("\nBody mass by species:")
print(penguins.groupby('species')['body_mass_g'].mean())

print("\nCorrelation with flipper length:")
print(f"r = {penguins['flipper_length_mm'].corr(penguins['body_mass_g']):.3f}")

In [ ]:
# 3. Multivariate: How do all measurements relate?
print("\nStep 3 - Multivariate Analysis:")
correlations_with_mass = penguins[numeric_cols].corr()['body_mass_g'].sort_values(ascending=False)
print("\nCorrelations with body mass:")
print(correlations_with_mass)

**CONCLUSION:** What predicts body mass?

1. Species is the strongest predictor
   - Gentoo: 5076g average
   - Chinstrap: 3733g average
   - Adelie: 3701g average
2. Among continuous variables, flipper length is best (r=0.87)
3. Bill measurements are weaker predictors (r=0.60 for length)
4. All measurements correlate (larger penguins are larger overall)

Next steps: Could build a model to predict mass from these features (prediction - INSY 7120) or use inferential statistics to test hypothesis.

Quick demonstration of inferential statistics (beyond our scope):


In [ ]:
# Brief demonstration: Statistical hypothesis testing with scipy
from scipy import stats

# Test 1: Do species differ significantly in body mass? (ANOVA)
adelie_mass = penguins[penguins['species']=='Adelie']['body_mass_g'].dropna()
chinstrap_mass = penguins[penguins['species']=='Chinstrap']['body_mass_g'].dropna()
gentoo_mass = penguins[penguins['species']=='Gentoo']['body_mass_g'].dropna()

f_stat, p_value = stats.f_oneway(adelie_mass, chinstrap_mass, gentoo_mass)
print(f"\n1. ANOVA: Do species differ in body mass?")
print(f"   F-statistic: {f_stat:.2f}, p-value: {p_value:.2e}")
print(f"   Result: {'Yes, significantly different' if p_value < 0.05 else 'No significant difference'} (p<0.05)")

# Test 2: Is flipper-mass correlation significant?
# Remove rows with missing values for both variables
valid_data = penguins[['flipper_length_mm', 'body_mass_g']].dropna()
r, p_value = stats.pearsonr(valid_data['flipper_length_mm'], valid_data['body_mass_g'])
print(f"\n2. Correlation test: Flipper length vs body mass")
print(f"   r = {r:.3f}, p-value: {p_value:.2e}")
print(f"   Result: {'Significant correlation' if p_value < 0.05 else 'Not significant'} (p<0.05)")


These tests confirm what we saw visually!

Statistical testing (scipy) is covered in depth elsewhere(?).

### Summary of Key Methods

Reference table for the methods we used:

| Analysis Type | Method | When to Use | Example |
|---------------|--------|-------------|---------|
| Continuous distribution | `hist()`, `describe()` | Univariate | Body mass distribution |
| Categorical distribution | `value_counts()` | Univariate | Species counts |
| Continuous relationship | `scatter()` + `corr()` | Bivariate | Flipper vs mass |
| Group comparison | `groupby()` + `boxplot()` | Bivariate | Mass by species |
| Category combinations | `pd.crosstab()` | Bivariate | Species × island |
| All numeric relationships | `corr()` + heatmap | Multivariate | Correlation matrix |
| Comprehensive view | `pairplot()` | Multivariate | All variables at once |
| Create categories | `pd.cut()`, `pd.qcut()` | Transform | Mass categories |
| Encode categories | `pd.get_dummies()` | Transform | Species as numbers |

### Key Takeaways

Remember:

- Start with questions, not techniques
- Univariate → Bivariate → Multivariate progression
- Visualize before quantifying
- Correlation ≠ causation (but generates hypotheses)
- Watch for Simpson's Paradox (aggregate patterns vs subgroup patterns)
- Transform data to answer specific questions
- Document your findings and reasoning

This systematic approach works for any dataset - apply it in your homework and final project!